In [ ]:
import pandas as pd
import numpy as np


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
article_path = "/content/drive/MyDrive/Development Data/dev_data_article.xlsx"
tweet_path = "/content/drive/MyDrive/Development Data/dev_data_tweet.xlsx"

PreProcessing


In [ ]:

df_tweet = pd.read_excel(tweet_path)
df_tweet=df_tweet.dropna()
df_tweet.columns = ['Id', 'text', 'class']
df_tweet.drop_duplicates(subset=['text'], inplace=True)
df_tweet[1570:1590]

,Id,text,class
2523,tweet_2524,"SBI दे रहा सबसे सस्ता Home Loan, 31 मार्च तक न...",0
2524,tweet_2525,RT @AshishK51455345: हर बड़े मुद्दे पर चीन को ...,0
2525,tweet_2526,"Santa ne bank loan pe 1 car li, Loan nahi chuk...",0
2526,tweet_2527,QT @Chamanlal_: सेंट्रल बैंक कश्मीरी गेट के ब्...,0
2530,tweet_2531,@srivatsayb Ghar banane ke liye tu ek bank se ...,0
2531,tweet_2532,RT @anupam0044: @AdvocateBLJain @capankajssm भ...,0
2532,tweet_2533,RT @birendra112016: @AdvocateBLJain @capankajs...,0
2533,tweet_2534,RT @ArunChe12360653: @SonuSood hi sir Mera naa...,0
2534,tweet_2535,@SonuSood Sir Aapse bahut help manga Apne medi...,0
2537,tweet_2538,Sir meine loan mein 1 lakh liya tha ab operati...,0


In [ ]:
df_article = pd.read_excel(article_path)
df_article.columns = ['Id', 'text','headline', 'class']
df_article = df_article.dropna()
df_article.drop_duplicates(subset=['text'] , inplace = True)
df_article.tail()


,Id,text,headline,class
3995,article_3996,All Hindi Bloggers List with All Detail\nMay 1...,All Hindi Bloggers List with All Detail,0
3996,article_3997,"Amazon Flex se Part Time me Earn kare ₹15,000 ...","Amazon Flex se Part Time me Earn kare ₹15,000 ...",0
3997,article_3998,Bad Link Building Strategies se Kaise Bachaye ...,Bad Link Building Strategies se Kaise Bachaye ...,0
3998,article_3999,Badiya Comment karte Samay Rakhe in Baat ka Dh...,Badiya Comment karte Samay Rakhe in Baat ka Dh...,0
3999,article_4000,Best Blogger Blog Theme का चुनाव कैसे करे ? [ ...,Best Blogger Blog Theme का चुनाव कैसे करे ? [ ...,0


In [ ]:

def clean_text(text):
  ans = []
  for tweet in text:
    tweet = tweet.lower()
    tweet = re.sub(r'[^\w\s]', '', tweet) 
    tweet = re.sub(r'^rt[\s]+','',tweet)
    tweet = re.sub(r'^qt[\s]+','',tweet)
    tweet = re.sub(r'https?:\/\/.*[\s]*','',tweet)
    tweet = re.sub(r'(#|@|:|>)',' ',tweet)
    tweet = re.sub(r'[\s]+',' ',tweet)
    # tweet = re.sub(r'[^\x00-\x7F]+|\x0c',' ', tweet) # remove all non-XML-compatible character ie the emojis
    ans.append(tweet[:4900])
  return ans


cleaned_text = clean_text(df_tweet['text'])
df_tweet['cleaned_text'] = cleaned_text

display(df_tweet)


cleaned_text = clean_text(df_article['text'])
df_article['cleaned_text'] = cleaned_text

display(df_article.head())




,Id,text,class,cleaned_text
0,tweet_0001,You'll 💜 my #PitchWars book if you like: 🦋 hat...,0,youll my pitchwars book if you like hatetolove...
1,tweet_0002,RT @SkySportsNews: 🚨 Breaking: #WBA have reach...,0,skysportsnews breaking wba have reached a verb...
2,tweet_0003,RT @stealyoman_cuso: really says a lot about s...,0,stealyoman_cuso really says a lot about societ...
3,tweet_0004,RT @PGtzsche1: HPV vaccines increased serious ...,0,pgtzsche1 hpv vaccines increased serious nervo...
4,tweet_0005,Ramaphosa says if you are positive you must se...,0,ramaphosa says if you are positive you must se...
...,...,...,...,...
3991,tweet_3992,#Realme ने 5G सेगमेंट में एक धांसू फोन #Realme...,1,realme न 5g सगमट म एक धस फन realmev15 लनच कय ह...
3992,tweet_3993,"भारत में लॉन्च हुआ Xiaomi Mi 10i, यह है 2021 क...",1,भरत म लनच हआ xiaomi mi 10i यह ह 2021 क सबस ससत...
3997,tweet_3998,चीनी मोबाइल कंपनी रियलमी ने अपना Realme V15 5G...,1,चन मबइल कपन रयलम न अपन realme v15 5g समरटफन लन...
3998,tweet_3999,QT @AnjaliSingh_IN: Aapne phone hi galat choos...,1,anjalisingh_in aapne phone hi galat choose kr ...


,Id,text,headline,class,cleaned_text
0,article_0001,Digitisation is one of the key buzzwords in th...,Pakistan’s digital landscape post-Covid,0,digitisation is one of the key buzzwords in th...
1,article_0002,Increase in tolerance limit up to 120 per cent...,Affordable housing gets shot in the arm,0,increase in tolerance limit up to 120 per cent...
2,article_0003,Home > News > World Sports News\n\nJonas Lossl...,Jonas Lossl leaves Everton to rejoin first clu...,0,home news world sports news jonas lossl leaves...
3,article_0004,"Source: Agfax.com\n\nBy Keith Brown, DTN Contr...",DTN Cotton Closing: Cotton Higher on Commoditi...,0,source agfaxcom by keith brown dtn contributin...
4,article_0005,"United Nations, Feb 4: The Serum Institute of ...","Serum Institute of India, UNICEF enter into lo...",0,united nations feb 4 the serum institute of in...


In [ ]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.hi import STOP_WORDS as STOP_WORDS_HI

nlp = spacy.load("en_core_web_sm")

In [ ]:

def preprocess_text(tweet):
  doc = nlp(tweet)
  token_list_for_tweet = [token for token in doc]
  hindi_removed = [token for token in doc if token.text not in STOP_WORDS_HI]
  filtered_tokens = [token for token in hindi_removed if not token.is_stop]
  lemmatized_tweet_list=[token.lemma_ for token in filtered_tokens if token.is_punct==False]
  return lemmatized_tweet_list

bag_of_words = []
for text in df_tweet['cleaned_text']:

  x = preprocess_text(text)
  bag_of_words.append(str(x))


df_tweet['bag_of_words'] = bag_of_words

In [ ]:
df_tweet

,Id,text,class,cleaned_text,bag_of_words
0,tweet_0001,You'll 💜 my #PitchWars book if you like: 🦋 hat...,0,youll my pitchwars book if you like hatetolove...,"['will', 'pitchwar', 'book', 'like', 'hatetolo..."
1,tweet_0002,RT @SkySportsNews: 🚨 Breaking: #WBA have reach...,0,skysportsnews breaking wba have reached a verb...,"['skysportsnews', 'break', 'wba', 'reach', 've..."
2,tweet_0003,RT @stealyoman_cuso: really says a lot about s...,0,stealyoman_cuso really says a lot about societ...,"['stealyoman_cuso', 'say', 'lot', 'society', '..."
3,tweet_0004,RT @PGtzsche1: HPV vaccines increased serious ...,0,pgtzsche1 hpv vaccines increased serious nervo...,"['pgtzsche1', 'hpv', 'vaccine', 'increase', 'n..."
4,tweet_0005,Ramaphosa says if you are positive you must se...,0,ramaphosa says if you are positive you must se...,"['ramaphosa', 'say', 'positive', 'self', 'isol..."
...,...,...,...,...,...
3991,tweet_3992,#Realme ने 5G सेगमेंट में एक धांसू फोन #Realme...,1,realme न 5g सगमट म एक धस फन realmev15 लनच कय ह...,"['realme', '5', 'g', 'सगमट', 'म', 'धस', 'फन', ..."
3992,tweet_3993,"भारत में लॉन्च हुआ Xiaomi Mi 10i, यह है 2021 क...",1,भरत म लनच हआ xiaomi mi 10i यह ह 2021 क सबस ससत...,"['भरत', 'म', 'लनच', 'हआ', 'xiaomi', 'mi', '10i..."
3997,tweet_3998,चीनी मोबाइल कंपनी रियलमी ने अपना Realme V15 5G...,1,चन मबइल कपन रयलम न अपन realme v15 5g समरटफन लन...,"['चन', 'मबइल', 'कपन', 'रयलम', 'अपन', 'realme',..."
3998,tweet_3999,QT @AnjaliSingh_IN: Aapne phone hi galat choos...,1,anjalisingh_in aapne phone hi galat choose kr ...,"['anjalisingh_in', 'aapne', 'phone', 'hi', 'ga..."


In [ ]:
bag_of_words = []
for text in df_article['cleaned_text']:

  x = preprocess_text(text)
  bag_of_words.append(str(x))


df_article['bag_of_words'] = bag_of_words

In [ ]:
df_article

,Id,text,headline,class,cleaned_text,bag_of_words
0,article_0001,Digitisation is one of the key buzzwords in th...,Pakistan’s digital landscape post-Covid,0,digitisation is one of the key buzzwords in th...,"['digitisation', 'key', 'buzzword', 'postcovid..."
1,article_0002,Increase in tolerance limit up to 120 per cent...,Affordable housing gets shot in the arm,0,increase in tolerance limit up to 120 per cent...,"['increase', 'tolerance', 'limit', '120', 'cen..."
2,article_0003,Home > News > World Sports News\n\nJonas Lossl...,Jonas Lossl leaves Everton to rejoin first clu...,0,home news world sports news jonas lossl leaves...,"['home', 'news', 'world', 'sports', 'news', 'j..."
3,article_0004,"Source: Agfax.com\n\nBy Keith Brown, DTN Contr...",DTN Cotton Closing: Cotton Higher on Commoditi...,0,source agfaxcom by keith brown dtn contributin...,"['source', 'agfaxcom', 'keith', 'brown', 'dtn'..."
4,article_0005,"United Nations, Feb 4: The Serum Institute of ...","Serum Institute of India, UNICEF enter into lo...",0,united nations feb 4 the serum institute of in...,"['united', 'nations', 'feb', '4', 'serum', 'in..."
...,...,...,...,...,...,...
3995,article_3996,All Hindi Bloggers List with All Detail\nMay 1...,All Hindi Bloggers List with All Detail,0,all hindi bloggers list with all detail may 15...,"['hindi', 'blogger', 'list', 'detail', '15', '..."
3996,article_3997,"Amazon Flex se Part Time me Earn kare ₹15,000 ...","Amazon Flex se Part Time me Earn kare ₹15,000 ...",0,amazon flex se part time me earn kare 15000 ea...,"['amazon', 'flex', 'se', 'time', 'earn', 'kare..."
3997,article_3998,Bad Link Building Strategies se Kaise Bachaye ...,Bad Link Building Strategies se Kaise Bachaye ...,0,bad link building strategies se kaise bachaye ...,"['bad', 'link', 'building', 'strategy', 'se', ..."
3998,article_3999,Badiya Comment karte Samay Rakhe in Baat ka Dh...,Badiya Comment karte Samay Rakhe in Baat ka Dh...,0,badiya comment karte samay rakhe in baat ka dh...,"['badiya', 'comment', 'karte', 'samay', 'rakhe..."


THEME CLASSIFIER

In [ ]:
# train_X , test_X ,train_Y , test_Y  = model_selection.train_test_split(df_tweet['bag_of_words'],df_tweet['class'],test_size=0.3)

a = df_tweet["bag_of_words"]
b = df_article["bag_of_words"]

print(a.shape)
print(b.shape)

df = pd.DataFrame(a.append(b, ignore_index = True))

display(df)

c = df_tweet["class"]
d = df_article["class"]
df_class = pd.DataFrame(c.append(d, ignore_index = True))

display(df_class)



(2400,)
(3567,)


,bag_of_words
0,"['will', 'pitchwar', 'book', 'like', 'hatetolo..."
1,"['skysportsnews', 'break', 'wba', 'reach', 've..."
2,"['stealyoman_cuso', 'say', 'lot', 'society', '..."
3,"['pgtzsche1', 'hpv', 'vaccine', 'increase', 'n..."
4,"['ramaphosa', 'say', 'positive', 'self', 'isol..."
...,...
5962,"['hindi', 'blogger', 'list', 'detail', '15', '..."
5963,"['amazon', 'flex', 'se', 'time', 'earn', 'kare..."
5964,"['bad', 'link', 'building', 'strategy', 'se', ..."
5965,"['badiya', 'comment', 'karte', 'samay', 'rakhe..."


,class
0,0
1,0
2,0
3,0
4,0
...,...
5962,0
5963,0
5964,0
5965,0


In [ ]:
Tfidf_vect = TfidfVectorizer()
Tfidf_vect.fit(df['bag_of_words'])
Train_X_Tfidf = Tfidf_vect.transform(df['bag_of_words'])

Train_X_Tfidf


# Test_X_Tfidf = Tfidf_vect.transform(test_X)

<5967x60252 sparse matrix of type '<class 'numpy.float64'>'
	with 633768 stored elements in Compressed Sparse Row format>

In [ ]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,df_class["class"])


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

Testing on Testdataset


In [ ]:
test_data_path = "/content/drive/MyDrive/Development Data/evaluation_data.xlsx"

In [ ]:
df_test = pd.read_excel(test_data_path)
df_test

,Text_ID,Text
0,article_4001,Market Highlights\n\nAn automotive constant ve...
1,article_4002,Overview:\n\nBe it as a means for food or as p...
2,article_4003,"GenScript USA Inc. (U.S.), Horizon Discovery G..."
3,article_4004,New Delhi: Prime Minister Narendra Modi spoke ...
4,article_4005,"New Delhi, Feb 1 (PTI) The Delhi High Court Mo..."
...,...,...
427,tweet_4206,#ios 6.1 #for #iphone #very #bad and #iPhone5 ...
428,tweet_4207,One #bad thing of #XiaomiRedmiNote4 is the au...
429,tweet_4208,I have a concern over the service provided . ...
430,tweet_4209,#Bought Nokia6.1Plus in Oct2018 and having wor...


In [ ]:
cleaned_text = clean_text(df_test['Text'])
df_test['cleaned_text'] = cleaned_text
# def shorten(text):
#   return text[:4900]

df_test["cleaned_text"]

0      market highlights an automotive constant veloc...
1      overview be it as a means for food or as pets ...
2      genscript usa inc us horizon discovery group p...
3      new delhi prime minister narendra modi spoke t...
4      new delhi feb 1 pti the delhi high court monda...
                             ...                        
427    ios 61 for iphone very bad and iphone5 not goo...
428    one bad thing of xiaomiredminote4 is the auto ...
429     i have a concern over the service provided pu...
430    bought nokia61plus in oct2018 and having worst...
431    now switching to oneplus_in oneplus6 really fr...
Name: cleaned_text, Length: 432, dtype: object

In [ ]:
df_test

,Text_ID,Text,cleaned_text
0,article_4001,Market Highlights\n\nAn automotive constant ve...,market highlights an automotive constant veloc...
1,article_4002,Overview:\n\nBe it as a means for food or as p...,overview be it as a means for food or as pets ...
2,article_4003,"GenScript USA Inc. (U.S.), Horizon Discovery G...",genscript usa inc us horizon discovery group p...
3,article_4004,New Delhi: Prime Minister Narendra Modi spoke ...,new delhi prime minister narendra modi spoke t...
4,article_4005,"New Delhi, Feb 1 (PTI) The Delhi High Court Mo...",new delhi feb 1 pti the delhi high court monda...
...,...,...,...
427,tweet_4206,#ios 6.1 #for #iphone #very #bad and #iPhone5 ...,ios 61 for iphone very bad and iphone5 not goo...
428,tweet_4207,One #bad thing of #XiaomiRedmiNote4 is the au...,one bad thing of xiaomiredminote4 is the auto ...
429,tweet_4208,I have a concern over the service provided . ...,i have a concern over the service provided pu...
430,tweet_4209,#Bought Nokia6.1Plus in Oct2018 and having wor...,bought nokia61plus in oct2018 and having worst...


In [ ]:
bag_of_words = []
for text in df_test['cleaned_text']:

  x = preprocess_text(text)
  bag_of_words.append(str(x))


df_test['bag_of_words'] = bag_of_words

In [ ]:
df_test["bag_of_words"]

0      ['market', 'highlight', 'automotive', 'constan...
1      ['overview', 'means', 'food', 'pet', 'animal',...
2      ['genscript', 'usa', 'inc', 'horizon', 'discov...
3      ['new', 'delhi', 'prime', 'minister', 'narendr...
4      ['new', 'delhi', 'feb', '1', 'pti', 'delhi', '...
                             ...                        
427    ['ios', '61', 'iphone', 'bad', 'iphone5', 'goo...
428    ['bad', 'thing', 'xiaomiredminote4', 'auto', '...
429    [' ', 'concern', 'service', 'provide', 'purcha...
430    ['buy', 'nokia61plus', 'oct2018', 'have', 'bad...
431    ['switch', 'oneplus_in', 'oneplus6', 'frustrat...
Name: bag_of_words, Length: 432, dtype: object

In [ ]:
Test_X_Tfidf = Tfidf_vect.transform(df_test["bag_of_words"])

Brand Extractor


In [ ]:
brand_list = ['Accer', 'Alcatel', 'AllView', 'Amazon', 'Amoi', 'Apple', 'Archos', 'Asus', 'AT&T', 'Benefon', 'BenQ', 'BenQ-Siemens', 'Bird', 'BlackBerry', 'BLU', 'Bosh', 'BQ', 'Casio', 'Cat', 'Celkon', 'Chea', 'CoolPad', 'Dell', 'Emporia', 'Energizer', 'Ericsson', 'Eten', 'Fujitsu Siemens', 'Garmin-Asus', 'Gigabyte', 'Gionee', 'Google', 'Haier', 'HP', 'HTC', 'Huawei', 'i-mate', 'i-mobile', 'Icemobile', 'Innostream', 'INQ', 'Intex', 'Jolla', 'Karbonn', 'Kyocera', 'Lava', 'LeEco', 'Lenovo', 'LG', 'Maxon', 'Maxwest', 'Meizu', 'Micromax', 'Microsoft', 'Mitac', 'Mitsubishi', 'Modu', 'Motorola', 'MWg', 'NEC', 'Neonode', 'NIU', 'Nokia', 'Nvidia', 'O2', 'OnePlus', 'Oppo', 'Orange', 'Palm', 'Panasonic', 'Pantech', 'Parla', 'Philips', 'Plum', 'Posh', 'Prestigio', 'QMobile', 'Qtek', 'Sagem', 'Samsung', 'Sendo', 'Sewon', 'Siemens', 'Sonim', 'Sony', 'Sony Ericsson', 'Spice', 'T-Mobile', 'Tel.Me.', 'Telit', 'Thuraya', 'Toshiba', 'Unnecto', 'Vertu', 'Verykook', 'Vivo', 'VK Mobile', 'Vodafone', 'Wiko', 'WND', 'Xcute', 'Xiaomi', 'XOLO', 'Yezz', 'Yota', 'Yu', 'ZTE']


In [ ]:
def extract_using_nlp(text):

    nlp_text = nlp(text)
    noun_chunks = nlp_text.noun_chunks
    # removing stop words and implementing word tokenization
    tokens = [token.text for token in nlp_text if not token.is_stop]
    # data = pd.read_csv(path) 
    
    # spec =  list(data['name'])

    spec = [item.lower() for item in brand_list]
    # print(spec)

    specset = []
    
    # check for one-grams (example: python)
    for token in tokens:
        if token.lower() in spec:
            specset.append(token)
    
    # check for bi-grams and tri-grams (example: machine learning)
    for token in noun_chunks:
        token = token.text.lower().strip()
        if token in spec:
            specset.append(token)
    
    return [i.capitalize() for i in set([i.lower() for i in specset])]
    
    
def find_organisation(text):
        
    
    doc = nlp(text)
    
    return [(X.text).capitalize() for X in doc.ents if X.label_ == "ORG"] 

In [ ]:


df_test["brands"] = df_test["cleaned_text"].apply(extract_using_nlp)




In [ ]:
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
i =0
index = list(df_test["Text_ID"])
for brand in df_test["brands"]:
  
  if brand != [] and predictions_SVM[i] == 0 and index[i][0] == 't':
    predictions_SVM[i] = 1
  i= i+1

predictions = list(predictions_SVM)
predictions


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


SENTIMENT AnalYSIS

In [ ]:
!pip install googletrans==4.0.0rc1

     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 1.2 MB 10.9 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
     |████████████████████████████████| 65 kB 3.4 MB/s 
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17415 sha256=8ad5e607802a757bda2beb3f21341d24d7d8b35f370253eaac5c95f39628a6b1
  Stored in directory: /root/.cache/pip/wheels/43/34/00/4fe71786ea6d12314b29037620c36d857e5d104ac2748bf82a
Successfully built googletrans


In [ ]:
!pip install flair

     |████████████████████████████████| 319 kB 5.4 MB/s 
     |████████████████████████████████| 19.7 MB 7.0 MB/s 
     |████████████████████████████████| 981 kB 36.2 MB/s 
     |████████████████████████████████| 2.9 MB 50.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 56 kB 4.5 MB/s 
     |████████████████████████████████| 788 kB 43.8 MB/s 
     |████████████████████████████████| 64 kB 2.4 MB/s 
     |████████████████████████████████| 1.2 MB 45.7 MB/s 
     |████████████████████████████████| 48 kB 5.1 MB/s 
     |████████████████████████████████| 62 kB 794 kB/s 
     |████████████████████████████████| 3.3 MB 36.3 MB/s 
     |████████████████████████████████| 895 kB 45.6 MB/s 
     |████████████████████████████████| 636 kB 47.8 MB/s 
     |████████████████████████████████| 109 kB 45.7 MB/s 
     |████████████████████████████████| 546 kB 46.4 MB/s 
  Created whe

In [ ]:
!pip install langdetect
!pip install google-trans-new

In [ ]:
from langdetect import detect
import time
from google_trans_new import google_translator  


translator = google_translator()  
def pleaseTranslate(text):
  # try:
  #     if detect(text) != 'en':
  #       text = translator.translate(text,lang_tgt='en') 
  # except :
  #       pass
  text = translator.translate(text,lang_tgt='en')
  return text

In [ ]:
def shorten(text):
  return text[:4900]
df_test["cleaned_text"] = df_test["cleaned_text"].apply(shorten)
# df_test["translated_text"] = df_test["cleaned_text"].apply(pleaseTranslate) 

In [ ]:
df_test[250:300]

,Text_ID,Text,cleaned_text,bag_of_words,brands
250,tweet_4029,RT @FatEmperor: Sweden is OLD NORMAL at the mo...,fatemperor sweden is old normal at the moment ...,"['fatemperor', 'sweden', 'old', 'normal', 'mom...",[]
251,tweet_4030,"RT @GurjitAFC: According to BBC Sport, West Br...",gurjitafc according to bbc sport west brom hav...,"['gurjitafc', 'accord', 'bbc', 'sport', 'west'...",[]
252,tweet_4031,RT @MailSport: West Brom agree deal to take Ai...,mailsport west brom agree deal to take ainsley...,"['mailsport', 'west', 'brom', 'agree', 'deal',...",[]
253,tweet_4032,RT @joseph_arruda: @thehill @PressSec The answ...,joseph_arruda thehill presssec the answer is n...,"['joseph_arruda', 'thehill', 'presssec', 'answ...",[]
254,tweet_4033,QT @farawe_kehinde: Liability 😏 ; @istolethetw...,farawe_kehinde liability istolethetweet yes an...,"['farawe_kehinde', 'liability', 'istolethetwee...",[]
255,tweet_4034,@schools_us @CBSNews @LoriLightfoot Parents ar...,schools_us cbsnews lorilightfoot parents are l...,"['schools_us', 'cbsnews', 'lorilightfoot', 'pa...",[]
256,tweet_4035,RT @SoulCity_SA: So why doesn’t everyone use c...,soulcity_sa so why doesnt everyone use condoms...,"['soulcity_sa', 'not', 'use', 'condom', 'secre...",[]
257,tweet_4036,RT @HHSGov: Are you wearing your #mask correct...,hhsgov are you wearing your mask correctly if ...,"['hhsgov', 'wear', 'mask', 'correctly', 'mask'...",[]
258,tweet_4037,RT @KiranMa17528987: *Blood group:* AB--Ve /A-...,kiranma17528987 blood group abve ave bve patie...,"['kiranma17528987', 'blood', 'group', 'abve', ...",[]
259,tweet_4038,@streetwiseprof @GuFinProf @drgoldstein @AlgoT...,streetwiseprof gufinprof drgoldstein algotrdr ...,"['streetwiseprof', 'gufinprof', 'drgoldstein',...",[]


In [ ]:
import flair

In [ ]:
flair_sentiment = flair.models.TextClassifier.load('en-sentiment')
def useFlair(text):
  s = flair.data.Sentence(text)
  flair_sentiment.predict(s)
  total_sentiment = s.labels
  return total_sentiment

2021-10-06 05:09:54,582 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_4.pt not found in cache, downloading to /tmp/tmp5ab2m1m0


100%|██████████| 265512723/265512723 [00:12<00:00, 21971070.40B/s]

2021-10-06 05:10:07,057 copying /tmp/tmp5ab2m1m0 to cache at /root/.flair/models/sentiment-en-mix-distillbert_4.pt


2021-10-06 05:10:07,730 removing temp file /tmp/tmp5ab2m1m0
2021-10-06 05:10:07,768 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
df_test["sentiment"] = df_test["cleaned_text"].apply(useFlair) 

In [ ]:
df_test

,Text_ID,Text,cleaned_text,bag_of_words,brands,sentiment
0,article_4001,Market Highlights\n\nAn automotive constant ve...,market highlights an automotive constant veloc...,"['market', 'highlight', 'automotive', 'constan...",[],[POSITIVE (0.9845)]
1,article_4002,Overview:\n\nBe it as a means for food or as p...,overview be it as a means for food or as pets ...,"['overview', 'means', 'food', 'pet', 'animal',...",[],[POSITIVE (0.6997)]
2,article_4003,"GenScript USA Inc. (U.S.), Horizon Discovery G...",genscript usa inc us horizon discovery group p...,"['genscript', 'usa', 'inc', 'horizon', 'discov...",[],[POSITIVE (0.9991)]
3,article_4004,New Delhi: Prime Minister Narendra Modi spoke ...,new delhi prime minister narendra modi spoke t...,"['new', 'delhi', 'prime', 'minister', 'narendr...",[],[POSITIVE (0.9139)]
4,article_4005,"New Delhi, Feb 1 (PTI) The Delhi High Court Mo...",new delhi feb 1 pti the delhi high court monda...,"['new', 'delhi', 'feb', '1', 'pti', 'delhi', '...",[Amazon],[NEGATIVE (0.9925)]
...,...,...,...,...,...,...
427,tweet_4206,#ios 6.1 #for #iphone #very #bad and #iPhone5 ...,ios 61 for iphone very bad and iphone5 not goo...,"['ios', '61', 'iphone', 'bad', 'iphone5', 'goo...",[],[NEGATIVE (1.0)]
428,tweet_4207,One #bad thing of #XiaomiRedmiNote4 is the au...,one bad thing of xiaomiredminote4 is the auto ...,"['bad', 'thing', 'xiaomiredminote4', 'auto', '...",[Xiaomi],[NEGATIVE (0.9996)]
429,tweet_4208,I have a concern over the service provided . ...,i have a concern over the service provided pu...,"[' ', 'concern', 'service', 'provide', 'purcha...",[Xiaomi],[NEGATIVE (0.9996)]
430,tweet_4209,#Bought Nokia6.1Plus in Oct2018 and having wor...,bought nokia61plus in oct2018 and having worst...,"['buy', 'nokia61plus', 'oct2018', 'have', 'bad...",[Nokia],[NEGATIVE (1.0)]


Headline Generator

In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('t5-base')
model = AutoModelWithLMHead.from_pretrained('t5-base', return_dict=True)

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:664: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [ ]:
def summarize(text):
  inputs = tokenizer.encode("summarize: " + text,
                          return_tensors='pt',
                          max_length=512,
                          truncation=True)
  summary_ids = model.generate(inputs, max_length=20, min_length=20, length_penalty=5, num_beams=2)
  summary = tokenizer.decode(summary_ids[0])
  return summary


In [ ]:
# df_test["Summary"] = df_test["cleaned_text"].apply(summarize)
summary = []
i =0 
for text in df_test["cleaned_text"]:
  if predictions_SVM[i] == 1:
    t = summarize(text)  
    t = re.sub(r'<pad>', '', t)  
    summary.append(t)
  else:
    summary.append("")
  
  i= i+1

df_test["Headline"] = summary

In [ ]:
# print(df_test["Headline"][0])
df_test

,Text_ID,Text,cleaned_text,bag_of_words,brands,sentiment,Headline
0,article_4001,Market Highlights\n\nAn automotive constant ve...,market highlights an automotive constant veloc...,"['market', 'highlight', 'automotive', 'constan...",[],[POSITIVE (0.9845)],
1,article_4002,Overview:\n\nBe it as a means for food or as p...,overview be it as a means for food or as pets ...,"['overview', 'means', 'food', 'pet', 'animal',...",[],[POSITIVE (0.6997)],
2,article_4003,"GenScript USA Inc. (U.S.), Horizon Discovery G...",genscript usa inc us horizon discovery group p...,"['genscript', 'usa', 'inc', 'horizon', 'discov...",[],[POSITIVE (0.9991)],
3,article_4004,New Delhi: Prime Minister Narendra Modi spoke ...,new delhi prime minister narendra modi spoke t...,"['new', 'delhi', 'prime', 'minister', 'narendr...",[],[POSITIVE (0.9139)],
4,article_4005,"New Delhi, Feb 1 (PTI) The Delhi High Court Mo...",new delhi feb 1 pti the delhi high court monda...,"['new', 'delhi', 'feb', '1', 'pti', 'delhi', '...",[Amazon],[NEGATIVE (0.9925)],
...,...,...,...,...,...,...,...
427,tweet_4206,#ios 6.1 #for #iphone #very #bad and #iPhone5 ...,ios 61 for iphone very bad and iphone5 not goo...,"['ios', '61', 'iphone', 'bad', 'iphone5', 'goo...",[],[NEGATIVE (1.0)],
428,tweet_4207,One #bad thing of #XiaomiRedmiNote4 is the au...,one bad thing of xiaomiredminote4 is the auto ...,"['bad', 'thing', 'xiaomiredminote4', 'auto', '...",[Xiaomi],[NEGATIVE (0.9996)],xiaomiredminote4 doesnt work at all xiaom
429,tweet_4208,I have a concern over the service provided . ...,i have a concern over the service provided pu...,"[' ', 'concern', 'service', 'provide', 'purcha...",[Xiaomi],[NEGATIVE (0.9996)],i have a concern over the service provided pu...
430,tweet_4209,#Bought Nokia6.1Plus in Oct2018 and having wor...,bought nokia61plus in oct2018 and having worst...,"['buy', 'nokia61plus', 'oct2018', 'have', 'bad...",[Nokia],[NEGATIVE (1.0)],nokia61plus has worst exp evn battery got die...


In [ ]:
def cleaner(col):
  result=[]
  # print(type())
  for text in col:
    print(type(col))
    text = str(text)
    text = text.lower()
    text = re.sub('\[','',text)
    text = re.sub('\]','',text)
    text = re.sub('\(.*\)','',text)
    #text = text[1:9]
    result.append(text)
    # print(text)
    return text
  

df_test["sentiment"] = df_test["sentiment"].apply(cleaner)
df_test["brands"] = df_test["brands"].apply(cleaner)
df_test

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

,Text_ID,Text,cleaned_text,bag_of_words,brands,sentiment,Headline
0,article_4001,Market Highlights\n\nAn automotive constant ve...,market highlights an automotive constant veloc...,"['market', 'highlight', 'automotive', 'constan...",None,positive,
1,article_4002,Overview:\n\nBe it as a means for food or as p...,overview be it as a means for food or as pets ...,"['overview', 'means', 'food', 'pet', 'animal',...",None,positive,
2,article_4003,"GenScript USA Inc. (U.S.), Horizon Discovery G...",genscript usa inc us horizon discovery group p...,"['genscript', 'usa', 'inc', 'horizon', 'discov...",None,positive,
3,article_4004,New Delhi: Prime Minister Narendra Modi spoke ...,new delhi prime minister narendra modi spoke t...,"['new', 'delhi', 'prime', 'minister', 'narendr...",None,positive,
4,article_4005,"New Delhi, Feb 1 (PTI) The Delhi High Court Mo...",new delhi feb 1 pti the delhi high court monda...,"['new', 'delhi', 'feb', '1', 'pti', 'delhi', '...",amazon,negative,
...,...,...,...,...,...,...,...
427,tweet_4206,#ios 6.1 #for #iphone #very #bad and #iPhone5 ...,ios 61 for iphone very bad and iphone5 not goo...,"['ios', '61', 'iphone', 'bad', 'iphone5', 'goo...",None,negative,
428,tweet_4207,One #bad thing of #XiaomiRedmiNote4 is the au...,one bad thing of xiaomiredminote4 is the auto ...,"['bad', 'thing', 'xiaomiredminote4', 'auto', '...",xiaomi,negative,xiaomiredminote4 doesnt work at all xiaom
429,tweet_4208,I have a concern over the service provided . ...,i have a concern over the service provided pu...,"[' ', 'concern', 'service', 'provide', 'purcha...",xiaomi,negative,i have a concern over the service provided pu...
430,tweet_4209,#Bought Nokia6.1Plus in Oct2018 and having wor...,bought nokia61plus in oct2018 and having worst...,"['buy', 'nokia61plus', 'oct2018', 'have', 'bad...",nokia,negative,nokia61plus has worst exp evn battery got die...


In [ ]:
df_test

,Text_ID,Text,cleaned_text,bag_of_words,brands,sentiment,Headline
0,article_4001,Market Highlights\n\nAn automotive constant ve...,market highlights an automotive constant veloc...,"['market', 'highlight', 'automotive', 'constan...",None,positive,
1,article_4002,Overview:\n\nBe it as a means for food or as p...,overview be it as a means for food or as pets ...,"['overview', 'means', 'food', 'pet', 'animal',...",None,positive,
2,article_4003,"GenScript USA Inc. (U.S.), Horizon Discovery G...",genscript usa inc us horizon discovery group p...,"['genscript', 'usa', 'inc', 'horizon', 'discov...",None,positive,
3,article_4004,New Delhi: Prime Minister Narendra Modi spoke ...,new delhi prime minister narendra modi spoke t...,"['new', 'delhi', 'prime', 'minister', 'narendr...",None,positive,
4,article_4005,"New Delhi, Feb 1 (PTI) The Delhi High Court Mo...",new delhi feb 1 pti the delhi high court monda...,"['new', 'delhi', 'feb', '1', 'pti', 'delhi', '...",amazon,negative,
...,...,...,...,...,...,...,...
427,tweet_4206,#ios 6.1 #for #iphone #very #bad and #iPhone5 ...,ios 61 for iphone very bad and iphone5 not goo...,"['ios', '61', 'iphone', 'bad', 'iphone5', 'goo...",None,negative,
428,tweet_4207,One #bad thing of #XiaomiRedmiNote4 is the au...,one bad thing of xiaomiredminote4 is the auto ...,"['bad', 'thing', 'xiaomiredminote4', 'auto', '...",xiaomi,negative,xiaomiredminote4 doesnt work at all xiaom
429,tweet_4208,I have a concern over the service provided . ...,i have a concern over the service provided pu...,"[' ', 'concern', 'service', 'provide', 'purcha...",xiaomi,negative,i have a concern over the service provided pu...
430,tweet_4209,#Bought Nokia6.1Plus in Oct2018 and having wor...,bought nokia61plus in oct2018 and having worst...,"['buy', 'nokia61plus', 'oct2018', 'have', 'bad...",nokia,negative,nokia61plus has worst exp evn battery got die...


In [ ]:

df_final = df_test[['Text_ID', 'Text', 'sentiment','Headline','brands']]
df_final["Mobile_Tech_Flag_Actual"] = [""]*432
df_final["Brands_Entity_Actual"] = [""]*432
df_final["Sentiment_Actual"] = [""]*432
df_final["Headline_Actual_Eng"] = [""]*432
df_final['Mobile_Tech_Flag_Predicted'] = predictions
df_final

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Text_ID,Text,sentiment,Headline,brands,Mobile_Tech_Flag_Actual,Brands_Entity_Actual,Sentiment_Actual,Headline_Actual_Eng,Mobile_Tech_Flag_Predicted
0,article_4001,Market Highlights\n\nAn automotive constant ve...,positive,,None,,,,,0
1,article_4002,Overview:\n\nBe it as a means for food or as p...,positive,,None,,,,,0
2,article_4003,"GenScript USA Inc. (U.S.), Horizon Discovery G...",positive,,None,,,,,0
3,article_4004,New Delhi: Prime Minister Narendra Modi spoke ...,positive,,None,,,,,0
4,article_4005,"New Delhi, Feb 1 (PTI) The Delhi High Court Mo...",negative,,amazon,,,,,0
...,...,...,...,...,...,...,...,...,...,...
427,tweet_4206,#ios 6.1 #for #iphone #very #bad and #iPhone5 ...,negative,,None,,,,,0
428,tweet_4207,One #bad thing of #XiaomiRedmiNote4 is the au...,negative,xiaomiredminote4 doesnt work at all xiaom,xiaomi,,,,,1
429,tweet_4208,I have a concern over the service provided . ...,negative,i have a concern over the service provided pu...,xiaomi,,,,,1
430,tweet_4209,#Bought Nokia6.1Plus in Oct2018 and having wor...,negative,nokia61plus has worst exp evn battery got die...,nokia,,,,,1


In [ ]:
df1 = df_final[["Text_ID","Mobile_Tech_Flag_Actual","Mobile_Tech_Flag_Predicted","Headline_Actual_Eng","Headline"]]
df1

,Text_ID,Mobile_Tech_Flag_Actual,Mobile_Tech_Flag_Predicted,Headline_Actual_Eng,Headline
0,article_4001,,0,,
1,article_4002,,0,,
2,article_4003,,0,,
3,article_4004,,0,,
4,article_4005,,0,,
...,...,...,...,...,...
427,tweet_4206,,0,,
428,tweet_4207,,1,,xiaomiredminote4 doesnt work at all xiaom
429,tweet_4208,,1,,i have a concern over the service provided pu...
430,tweet_4209,,1,,nokia61plus has worst exp evn battery got die...


In [ ]:
df1.columns = ["Text_ID","Mobile_Tech_Flag_Actual","Mobile_Tech_Flag_Predicted","Headline_Actual_Eng","Headline_Generated_Eng_Lang"]
df1

,Text_ID,Mobile_Tech_Flag_Actual,Mobile_Tech_Flag_Predicted,Headline_Actual_Eng,Headline_Generated_Eng_Lang
0,article_4001,,0,,
1,article_4002,,0,,
2,article_4003,,0,,
3,article_4004,,0,,
4,article_4005,,0,,
...,...,...,...,...,...
427,tweet_4206,,0,,
428,tweet_4207,,1,,xiaomiredminote4 doesnt work at all xiaom
429,tweet_4208,,1,,i have a concern over the service provided pu...
430,tweet_4209,,1,,nokia61plus has worst exp evn battery got die...


In [ ]:
df2 = df_final[["Text_ID","Mobile_Tech_Flag_Actual","Mobile_Tech_Flag_Predicted","Brands_Entity_Actual","Sentiment_Actual","brands","sentiment"]]
df2.columns = ["Text_ID","Mobile_Tech_Flag_Actual","Mobile_Tech_Flag_Predicted","Brands_Entity_Actual","Sentiment_Actual", "Brands_Entity_Identified", "Sentiment_Identified"]
df2

,Text_ID,Mobile_Tech_Flag_Actual,Mobile_Tech_Flag_Predicted,Brands_Entity_Actual,Sentiment_Actual,Brands_Entity_Identified,Sentiment_Identified
0,article_4001,,0,,,None,positive
1,article_4002,,0,,,None,positive
2,article_4003,,0,,,None,positive
3,article_4004,,0,,,None,positive
4,article_4005,,0,,,amazon,negative
...,...,...,...,...,...,...,...
427,tweet_4206,,0,,,None,negative
428,tweet_4207,,1,,,xiaomi,negative
429,tweet_4208,,1,,,xiaomi,negative
430,tweet_4209,,1,,,nokia,negative


In [ ]:
b = list(df2["Brands_Entity_Identified"])
i = 0
sent_list = []
for sentiment in df2["Sentiment_Identified"]:
  if b[i] == None:
     sent_list.append("")
  else:
    sent_list.append(sentiment)
  i = i + 1

df2["Sentiment_Identified"] = sent_list
df2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,Text_ID,Mobile_Tech_Flag_Actual,Mobile_Tech_Flag_Predicted,Brands_Entity_Actual,Sentiment_Actual,Brands_Entity_Identified,Sentiment_Identified
0,article_4001,,0,,,None,
1,article_4002,,0,,,None,
2,article_4003,,0,,,None,
3,article_4004,,0,,,None,
4,article_4005,,0,,,amazon,negative
...,...,...,...,...,...,...,...
427,tweet_4206,,0,,,None,
428,tweet_4207,,1,,,xiaomi,negative
429,tweet_4208,,1,,,xiaomi,negative
430,tweet_4209,,1,,,nokia,negative


In [ ]:
def clearNone(text):
  if text==None:
    return ""
  else:
    return text

df2["Brands_Entity_Identified"] = df2["Brands_Entity_Identified"].apply(clearNone)
df2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,Text_ID,Mobile_Tech_Flag_Actual,Mobile_Tech_Flag_Predicted,Brands_Entity_Actual,Sentiment_Actual,Brands_Entity_Identified,Sentiment_Identified
0,article_4001,,0,,,,
1,article_4002,,0,,,,
2,article_4003,,0,,,,
3,article_4004,,0,,,,
4,article_4005,,0,,,amazon,negative
...,...,...,...,...,...,...,...
427,tweet_4206,,0,,,,
428,tweet_4207,,1,,,xiaomi,negative
429,tweet_4208,,1,,,xiaomi,negative
430,tweet_4209,,1,,,nokia,negative


In [ ]:


df1 = df1.set_index("Text_ID")
df2.reset_index(drop=True, inplace=True)
df2 = df2.set_index("Text_ID")
df1

,Mobile_Tech_Flag_Actual,Mobile_Tech_Flag_Predicted,Headline_Actual_Eng,Headline_Generated_Eng_Lang
Text_ID,,,,
article_4001,,0,,
article_4002,,0,,
article_4003,,0,,
article_4004,,0,,
article_4005,,0,,
...,...,...,...,...
tweet_4206,,0,,
tweet_4207,,1,,xiaomiredminote4 doesnt work at all xiaom
tweet_4208,,1,,i have a concern over the service provided pu...


In [ ]:
from google.colab import files

df1.to_csv('output_1.csv')
df2.to_csv('output_2.csv')
# files.download('output_1.csv')
# files.download('output_2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
model_save_name = 'classifier.pt'
path = F"/content/drive/My Drive/{model_save_name}" 
torch.save(model.state_dict(), path)

In [ ]:
model_save_name = 'classifier.pt'
path = F"/content/drive/My Drive/{model_save_name}"
model.load_state_dict(torch.load(path))

<All keys matched successfully>